In [1]:
pip install opencv-python numpy

In [2]:
import cv2
import numpy as np


resim_yolu = 'resim.png' 

# Görüntüyü yükleme
img = cv2.imread(resim_yolu)

if img is None:
    # Resim bulunamazsa hata mesajı
    print(f"Hata: Resim yolu '{resim_yolu}' bulunamadı. Dosya adını ve klasörü kontrol edin.")
else:
    #  Görüntüyü bir pencerede göster
    cv2.imshow('Orijinal Resim', img)
    
    #  Resmin boyutlarını ve kanal sayısını yazdır
    # img.shape formatı: (Yükseklik, Genişlik, Kanal)
    print(f"Görüntü Boyutu (Yükseklik, Genişlik, Kanal): {img.shape}")
    print(f"Veri Tipi: {img.dtype}") # Piksel değerlerinin tipi (genellikle uint8)
    
    # Pencerenin açık kalmasını sağlar (herhangi bir tuşa basılana kadar)
    cv2.waitKey(0) 
    cv2.destroyAllWindows() # Tüm OpenCV pencerelerini kapat

Görüntü Boyutu (Yükseklik, Genişlik, Kanal): (1024, 1536, 3)
Veri Tipi: uint8


In [3]:
# Görüntüyü B, G, R kanallarına ayır
B, G, R = cv2.split(img)

# Kanalları ayrı ayrı görüntüleme
# NOT: Her kanal sadece yoğunluk verisi içerdiği için siyah beyaz görünür.
cv2.imshow('Mavi Kanal', B)
cv2.imshow('Yesil Kanal', G)
cv2.imshow('Kirmizi Kanal', R)

print("Görüntü 3 renk kanalına (B, G, R) ayrıldı ve görüntülendi.")

# Pencereyi kapatmak için tuş bekleniyor
cv2.waitKey(0) 
cv2.destroyAllWindows() 


# Kanal Takası Yapılmış Versiyon Oluşturma (R ve B'yi değiştirme)
# OpenCV BGR kullanır. Biz RGB sırasıyla birleştirerek renkleri değiştiriyoruz.
img_swapped = cv2.merge([R, G, B]) 
cv2.imshow('Kanal Takasi (Kirmizi ve Mavi Yer Degistirdi)', img_swapped)

print("Kanal takası başarıyla yapıldı.")

# Pencereyi kapatmak için tuş bekleniyor
cv2.waitKey(0) 
cv2.destroyAllWindows()

Görüntü 3 renk kanalına (B, G, R) ayrıldı ve görüntülendi.
Kanal takası başarıyla yapıldı.


In [ ]:
# Resim boyutları: Yükseklik: 1080, Genişlik: 1920

# Yükseklik (Y ekseni: 0'dan 1080'e kadar)
y_baslangic = 100 
y_bitis = 850      

# Genişlik (X ekseni: 0'dan 1920'ye kadar)
x_baslangic = 50  
x_bitis = 1400   

# 5a. Görüntüyü kırpma

img_cropped = img[y_baslangic:y_bitis, x_baslangic:x_bitis]
cv2.imshow('Kirpilmis Resim (Merkez 500x500)', img_cropped)

# 5b. Görüntüyü yeniden boyutlandırma (bu kısım sabit kaldı, hedef boyut 300x400)
yeni_boyut = (300, 400) 
img_resized = cv2.resize(img, yeni_boyut)
cv2.imshow('Yeniden Boyutlandirilmis Resim (300x400)', img_resized)

print(f"Resim ({y_baslangic}:{y_bitis}, {x_baslangic}:{x_bitis}) aralığında kırpıldı ve {yeni_boyut} boyutuna yeniden ayarlandı.")

# Pencereyi kapatmak için tuş bekleniyor
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
import cv2
import numpy as np

# Web kamerasını aç (0, genellikle varsayılan kameradır) [cite: 24]
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Hata: Kamera açılamadı! Lütfen başka bir programın kamerayı kullanmadığından emin olun.")
else:
    print("Kamera başarıyla açıldı. Video akışını durdurmak için 'q' tuşuna basın.")
    
    while True:
        # Kameradan bir kare oku
        ret, frame = cap.read()
        
        if not ret:
            print("Kare alınamadı, çıkılıyor...")
            break
            
        # ---  GÖREV: Kare Üzerine Şekiller ve Metin Çizme  ---
        
        #  Yeşil bir dikdörtgen çizme (BGR: (0, 255, 0))
        cv2.rectangle(frame, (50, 50), (250, 250), (0, 255, 0), 3)
        
        #  Mavi bir daire çizme (içi dolu)
        cv2.circle(frame, (400, 150), 100, (255, 0, 0), -1) 
        
        # Beyaz metin ekleme
        cv2.putText(frame, 'CANLI VIDEO', (50, 450), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)

        # Görüntüyü bir pencerede göster
        cv2.imshow('CANLI VIDEO WIZARD', frame)
        
        # Kullanıcı girişini bekle
        key = cv2.waitKey(1) 
        
        # 'q' tuşu ile çıkış
        if key == ord('q'): 
            break
        
        # GÖREV: Ekran Görüntüsü (Screenshot) Özelliği 
        # 's' tuşuna basılırsa kareyi kaydet
        elif key == ord('s'):
            cv2.imwrite('screenshot_proje2.png', frame)
            print("Ekran görüntüsü kaydedildi: screenshot_proje2.png")

    # Tüm kaynakları serbest bırak ve pencereleri kapat
    cap.release()
    cv2.destroyAllWindows()

Kamera başarıyla açıldı. Video akışını durdurmak için 'q' tuşuna basın.


In [6]:
import cv2
import numpy as np

# Global değişkenler ve callback fonksiyonu (Değişmedi)
drawing = False 
last_x, last_y = -1, -1 

def cizim_yap(event, x, y, flags, param):
    """Fare olaylarını işleyen geri çağrı fonksiyonu"""
    global drawing, last_x, last_y
   
    
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        last_x, last_y = x, y
        
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing == True:
            
            cv2.circle(param, (x, y), 5, (0, 0, 255), -1) 
            last_x, last_y = x, y 
            
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False

# Ana Program ve Kamerayı Bağlama
cap = cv2.VideoCapture(0)

#  ÖNCE PENCEREYİ OLUŞTUR
pencere_adi = 'Boyama Programi'
cv2.namedWindow(pencere_adi) 

# SONRA FARE OLAYLARINI PENCEREYE BAĞLA

cv2.setMouseCallback(pencere_adi, cizim_yap, None) 

if not cap.isOpened():
    print("Hata: Kamera açılamadı.")
else:
    print("Canlı yayın başladı. Çizim yapmak için fare sol tuşuna basılı tutun. Çıkış için 'q'.")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
       
        cv2.setMouseCallback(pencere_adi, cizim_yap, frame) # Bu satır, çizimlerin güncel kareye yapılmasını sağlar.

        cv2.imshow(pencere_adi, frame)
        
        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

Canlı yayın başladı. Çizim yapmak için fare sol tuşuna basılı tutun. Çıkış için 'q'.


In [7]:
import cv2
import numpy as np

# GÖREV: Takip Edilecek Rengi Belirleme 
# ÖNEMLİ: Kendi takip etmek istediğiniz nesnenin 

low_blue = np.array([90, 50, 50])  # Düşük Ton, Doygunluk, Değer
high_blue = np.array([130, 255, 255]) # Yüksek Ton, Doygunluk, Değer

# Kamerayı aç
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Hata: Kamera açılamadı.")
else:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
      
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        
        mask = cv2.inRange(hsv, low_blue, high_blue) # [cite: 33]
        
        
        res = cv2.bitwise_and(frame, frame, mask=mask)

        # GÖREV: Konturları (Nesne Sınırlarını) Bulma
        
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # [cite: 33]

        # 3. ve 4. GÖREV: Filtreleme ve Bounding Box (Sınırlayıcı Kutu) Çizme
        if contours:
             
            largest_contour = max(contours, key=cv2.contourArea) # [cite: 33]
            
            #  GÖREV: Gürültüyü/küçük nesneleri filtrele
            if cv2.contourArea(largest_contour) > 500: 
                
                #  GÖREV: En büyük nesnenin etrafına sınırlayıcı kutu çiz
               
                x, y, w, h = cv2.boundingRect(largest_contour)
                
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                
                cv2.putText(frame, 'NESNE ALGILANDI', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

        # Sonuçları göster
        cv2.imshow('Orijinal Kare (Takip)', frame)
        cv2.imshow('HSV Maskesi', mask) # Bu pencere sadece beyaz/siyah (ikili) maskeyi gösterir
        
        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [12]:
import cv2
import numpy as np

# Haar Cascade dosyasını yükle

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

if face_cascade.empty():
    # Bu kontrol, dosyanın hâlâ bulunamadığı durumlar için kalmalı
    print("Hata: Haar Cascade XML dosyası yüklenemedi. Lütfen 'haarcascade_frontalface_default.xml' dosyasının doğru klasörde olduğundan emin olun.")
else:
    # Kamerayı aç
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Hata: Kamera açılamadı.")
    else:
        print("Canlı yüz algılama başladı. Yüzünüzü kameraya yaklaştırın. Çıkış için 'q'.")

        while True:
            ret, frame = cap.read()
            if not ret:
                break
                
            # Performans için kareyi gri tonlamaya çevir
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            # Yüzleri algıla
            faces = face_cascade.detectMultiScale(
                gray, 
                scaleFactor=1.1, 
                minNeighbors=5,  
                minSize=(30, 30) 
            )
            
            # Yüz Algılama ve Filtre Uygulama
            for (x, y, w, h) in faces:
                # 1. Yüz Çevresine Kutu Çiz
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                
                # 2. GÖREV: Yüz Bölgesine "Privacy Blur" (Gizlilik Bulanıklığı) Uygula
                face_roi = frame[y:y+h, x:x+w]
                
                # Bulanıklık uygula (çekirdek boyutu ne kadar büyükse o kadar bulanık)
                # w//8 | 1, genişliği 8'e bölüp tek sayıya yuvarlar (Gaussian blur için gerekli)
                blur_amount = w // 10 | 1 # Bulanıklık miktarını yüz boyutuna göre ayarlar
                face_roi = cv2.GaussianBlur(face_roi, (blur_amount, blur_amount), 0)
                
                # Bulanık bölgeyi tekrar kareye yerleştir
                frame[y:y+h, x:x+w] = face_roi
                
            # 3. GÖREV: Yüz Sayısını Görüntüleme
            face_count = len(faces)
            cv2.putText(frame, f'Yuz Sayisi: {face_count}', (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            cv2.imshow('Akilli Selfie Uygulamasi', frame)
            
            if cv2.waitKey(1) == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

Canlı yüz algılama başladı. Yüzünüzü kameraya yaklaştırın. Çıkış için 'q'.


In [13]:
import cv2
import numpy as np

# 1. GÖREV: Sanal İçeriği Yükle
logo = cv2.imread('logo.png') 
if logo is None:
    print("Hata: 'logo.png' bulunamadı. Lütfen sanal görüntüyü klasöre kopyalayın.")
    # Logo bulunamazsa programın çökmemesi için boş bir resim oluştur
    logo = np.zeros((300, 300, 3), dtype=np.uint8) 

# Marker'ı takip etmek için HSV aralığı (Örnek: Parlak Yeşil bir kağıt)
# Kendi markerinizin rengine göre bu değerleri AYARLAMALISINIZ!
# Örnek: Yeşil renk için HSV aralığı
low_marker = np.array([35, 100, 100])  
high_marker = np.array([80, 255, 255]) 

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, low_marker, high_marker)
    
    # Konturları bul (Marker'ı tespit et)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest_contour = max(contours, key=cv2.contourArea)

        # 2. GÖREV: Marker'ın Köşelerini Bulma (Yaklaşık Dörtgen Algılama)
        if cv2.contourArea(largest_contour) > 2000:
            # Konturdan yaklaşımla köşeleri bul (Poligonal Yaklaşım)
            epsilon = 0.04 * cv2.arcLength(largest_contour, True) # Kontur uzunluğunun %4'ü hassasiyet
            approx = cv2.approxPolyDP(largest_contour, epsilon, True)

            # Sadece DÖRT köşe bulursak (kare veya dikdörtgen marker) devam et
            if len(approx) == 4:
                
                # Köşeleri düzgün sırala (bu, dönüşümün doğru yapılması için kritiktir)
                approx = approx.reshape((4, 2))
                # Basit bir sıralama fonksiyonu (Genellikle köşe bulma algoritmaları bu sıralamayı basitleştirir)                
                # Köşeleri al: approx[0][0], approx[1][0], ...
                
                # Sanal Görüntünün Köşeleri (Logo'nun Düzlemsel Hali)
                src_pts = np.float32([
                    [0, 0],                            # Sol Üst
                    [logo.shape[1] - 1, 0],           # Sağ Üst
                    [logo.shape[1] - 1, logo.shape[0] - 1], # Sağ Alt
                    [0, logo.shape[0] - 1]             # Sol Alt
                ])
                
                # Marker'ın Köşeleri (Gerçekteki Eğik Konumu)
                # approx[0][0]'dan itibaren sıralama (örneğin Sol Üst, Sağ Üst, Sağ Alt, Sol Alt) varsayılır.
                dst_pts = np.float32([approx[0], approx[1], approx[2], approx[3]]) 
                
                # 3. GÖREV: Perspektif Dönüşümü Matrisini Hesapla
                M = cv2.getPerspectiveTransform(src_pts, dst_pts)
                
                # 4. GÖREV: Sanal görüntüyü marker'ın perspektifine dönüştür
                warped_logo = cv2.warpPerspective(logo, M, (frame.shape[1], frame.shape[0]))
                
                # 5. GÖREV: Dönüşmüş Logoyu Canlı Kare Üzerine Yerleştirme
                
                mask_logo = np.zeros(frame.shape, dtype=np.uint8)
                cv2.fillConvexPoly(mask_logo, approx.astype(int), (255, 255, 255), cv2.LINE_AA)
                mask_logo_inv = cv2.bitwise_not(mask_logo)
                
                # Orijinal karede logo'nun geleceği bölgeyi temizle
                frame_masked = cv2.bitwise_and(frame, mask_logo_inv)
                
                # Logo'yu temizlenmiş karenin üzerine ekle
                final_output = cv2.add(frame_masked, warped_logo)
                frame = final_output # Ana kareyi güncelle

                # Marker'ın etrafına yeşil bir kutu çizerek marker'ı görselleştir
                cv2.polylines(frame, [approx], True, (0, 255, 0), 2)
                
    cv2.imshow('Artirilmis Gerceklik (AR) Proje 6', frame)
    
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()